## Train Ranking Model

In this notebook, we will train a ranking model using gradient boosted trees. 

Let's start by loading the datasets we created in the previous notebook.

In [1]:
import pandas as pd

X_train = pd.read_csv("ranking_train_df.csv")
X_val = pd.read_csv("ranking_val_df.csv")
y_train = X_train.pop("label")
y_val = X_val.pop("label")

X_train.sample(5)

,age,month_sin,month_cos,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,...,user_emb_6,user_emb_7,user_emb_8,user_emb_9,user_emb_10,user_emb_11,user_emb_12,user_emb_13,user_emb_14,user_emb_15
4237739,35.0,-0.500000,-0.866025,Sweater,Garment Upper body,Stripe,Off White,Dusty Light,White,Young Boy Knitwear,...,-1.413316,-0.851314,0.012765,1.747324,1.302497,0.685798,-0.073758,-1.121989,0.006765,0.674256
2125535,24.0,-0.500000,-0.866025,Shirt,Garment Upper body,Check,Dark Blue,Dark,Blue,Tops Woven,...,-0.202677,0.194127,-0.006056,0.287214,1.039907,0.482088,-0.980501,-0.576162,0.126563,-0.170898
3686644,23.0,-0.866025,-0.500000,Blouse,Garment Upper body,Solid,Dark Green,Medium,Green,Blouse & Dress,...,-0.873762,0.618183,0.561952,1.204167,0.799343,0.016992,-1.268029,0.130244,-0.745781,1.146533
460797,52.0,-0.866025,-0.500000,Skirt,Garment Lower body,Solid,Black,Dark,Black,Skirt,...,-0.864712,1.184796,0.037965,0.092006,1.633265,0.252163,-1.409775,0.023782,-0.009513,0.622915
3791724,50.0,0.500000,0.866025,Necklace,Accessories,Solid,Gold,Bright,Metal,Jewellery,...,-0.560713,-0.028593,-0.791156,0.835248,0.538206,-0.278036,1.069142,-0.221891,-0.441997,-0.002153


Let's train a model.

In [2]:
from catboost import CatBoostClassifier, Pool

cat_features = list(
    X_train.select_dtypes(include=['string', 'object']).columns
)

pool_train = Pool(X_train, y_train, cat_features=cat_features)
pool_val = Pool(X_val, y_val, cat_features=cat_features)

model = CatBoostClassifier(
    learning_rate=0.2,
    iterations=100,
    depth=10,
    scale_pos_weight=10,
    early_stopping_rounds=5,
    use_best_model=True
)

model.fit(pool_train, eval_set=pool_val)

0:	learn: 0.6511688	test: 0.6609086	best: 0.6609086 (0)	total: 3.46s	remaining: 5m 42s
1:	learn: 0.6150501	test: 0.6338675	best: 0.6338675 (1)	total: 6.76s	remaining: 5m 31s
2:	learn: 0.5845732	test: 0.6124480	best: 0.6124480 (2)	total: 9.55s	remaining: 5m 8s
3:	learn: 0.5633841	test: 0.5988707	best: 0.5988707 (3)	total: 12.2s	remaining: 4m 53s
4:	learn: 0.5422082	test: 0.5849703	best: 0.5849703 (4)	total: 15.9s	remaining: 5m 2s
5:	learn: 0.5270976	test: 0.5760256	best: 0.5760256 (5)	total: 18.5s	remaining: 4m 49s
6:	learn: 0.5181974	test: 0.5715440	best: 0.5715440 (6)	total: 20.8s	remaining: 4m 36s
7:	learn: 0.5078444	test: 0.5656371	best: 0.5656371 (7)	total: 23.2s	remaining: 4m 26s
8:	learn: 0.4980965	test: 0.5602779	best: 0.5602779 (8)	total: 25.7s	remaining: 4m 19s
9:	learn: 0.4883098	test: 0.5553846	best: 0.5553846 (9)	total: 28.3s	remaining: 4m 15s
10:	learn: 0.4813042	test: 0.5521312	best: 0.5521312 (10)	total: 31s	remaining: 4m 10s
11:	learn: 0.4719715	test: 0.5474191	best: 0.

Next, we'll evaluate how well the model performs on the validation data.

In [3]:
from sklearn.metrics import classification_report

preds = model.predict(pool_val)

print(classification_report(y_val, preds))

              precision    recall  f1-score   support

           0       0.96      0.79      0.87   1756530
           1       0.24      0.69      0.36    175653

    accuracy                           0.78   1932183
   macro avg       0.60      0.74      0.61   1932183
weighted avg       0.90      0.78      0.82   1932183



It can be seen that the model has a low F1-score on the positive class (higher is better). The performance could potentially be improved by adding more features to the dataset, e.g. image embeddings.

Let's see which features our model considers important.

In [4]:
feat_to_score = {feature: score for feature, score in zip(
    X_train.columns, model.feature_importances_)}

feat_to_score = dict(
    sorted(
        feat_to_score.items(),
        key=lambda item: item[1],
        reverse=True
    )
)

feat_to_score


{'item_emb_8': 6.822848768403623,
 'item_emb_1': 5.1056785624608985,
 'item_emb_0': 5.06612408081166,
 'user_emb_9': 5.062247048422682,
 'user_emb_0': 5.059393722266358,
 'item_emb_14': 5.002919839900571,
 'item_emb_9': 4.618121922226104,
 'user_emb_14': 4.333259492550345,
 'user_emb_1': 4.077260315080454,
 'item_emb_4': 3.706886996224119,
 'item_emb_7': 3.433798739920005,
 'item_emb_5': 3.3971001987178617,
 'user_emb_7': 3.38884717928238,
 'user_emb_4': 3.1531758739541944,
 'user_emb_5': 3.0499661001664062,
 'item_emb_2': 2.7574471992719864,
 'user_emb_2': 2.681924640000333,
 'user_emb_8': 2.5640059218820017,
 'user_emb_3': 2.446951229350053,
 'item_emb_12': 2.0553585608760034,
 'item_emb_3': 2.048986871022075,
 'user_emb_11': 1.9560705811791053,
 'user_emb_12': 1.9104046895184512,
 'item_emb_11': 1.880350125795843,
 'user_emb_6': 1.8707347439786308,
 'month_sin': 1.809736945572611,
 'item_emb_6': 1.8085550077836445,
 'item_emb_10': 1.7656449339781786,
 'item_emb_15': 1.66061233496592

It can be seen that the model places high importance on user and item embedding features. Consequently, better trained embeddings could yield a better ranking model.

Finally, we'll save our model.

In [5]:
model.save_model("ranking_model.cbm")

### Next Steps

Now we have trained both a retrieval and a ranking model, which will allow us to generate recommendations for users. In the next notebook, we'll take a look at how we can deploy these models with the `HSML` library.